# DEVNET-2449 Python for Enterprise Network Elements
## Automation exploration with Python

## Jupyter Notebooks
The *Jupyter Notebooks* in this workbench class are interactive learning environments. We are using them to learn about network device programability in combination with virtual Cisco network devices.

Notebooks are **interactive**, the code and text in each notebook can be changed on the fly and the result of those changes can be seen immediately within the notebook itself.

Notebooks are organized in *cells*, each displayed *rectangle* or *cell* is basically either Markdown text (like this cell) or Python code. Python code can be *executed* by either clicking **Run Cell, select below** in the 'Cell' menu in the menu bar or the key combination 'Shift-Enter'.

<p>**IMPORTANT for Execution. PLEASE READ**. At its simplest, this is a playbook and execution is top-down. Code cells will look something like `In [#]:` (with '#' being a number). That's where you want to single-click. To execute the code, simply hit Shift+Enter.</p>

<p>The code should execute, and move onto the next cell. The number in `In [#]:` will change and if there is some return value (as opposed to an explicit `print()` statement) it will appear in an `Out [#]:` cell. The numbers will change after each cell execution.</p>

<p>It's easy to "execute" an entire notebook, task-by-task, by continuing to execute Shift+Return. Also, if you ever see `In [*]:` in your code cells (note the asterisk '\*'), it just means things are executing, and you are probably waiting on data to come back.</p>

<p>So depending on what you're doing, some patience might be required.</p>

</div>


## Environment
The workbench environment is running 100% locally on provided Macbooks. We use [Vagrant](https://www.vagrantup.com/) to run two [CSR 1000V virtual routers](http://www.cisco.com/c/en/us/products/routers/cloud-services-router-1000v-series/index.html) running IOS XE 16.6 and we use [Docker container](https://www.docker.com/) to run [Jupyter](http://jupyter.org/) interactive notebooks (like the document you are currently looking at).

The Python environment in the notebooks communicate with the routers via a host-only *internal* network on the MacBook Pro. The MacBook has its own IP address on this host-only network: `172.20.20.1`. The container can access the routers on their own IP addresses, `172.20.20.10` and `172.20.20.20` respectively. The following diagram shows a high level overview of the environment:

![Topology](images/topology.png)

We can access the CLI of the CSR1000v using a terminal (`Terminal.app`) and, while in the Vagrant directory, typing `vagrant ssh R1` or `vagrant ssh R2`. Because a public key has been installed into the router(s), we get a privileged shell / CLI without being prompted for a password.

There are some static routes configured as well as OSPF between routers.

In [1]:
# Let's first define connection parameters
HOST = '172.20.20.10'
USERNAME = 'vagrant'
PASSWORD = 'vagrant'
ENABLE_PASSWORD = 'cisco'

### Introducing Netmiko
Netmiko is Python module written by Kirk Byers which allows to interact with a network device via SSH. It automatically handles prompts and provides simple API for the most common operations.

In [2]:
from netmiko import ConnectHandler
from pprint import pprint

def get_output(cli_command_list):
    """Gets output from CLI command list executed the device
    
    This function connects to the device via netmiko module, gets outputs from all commands and closes SSH
    session.
    
    Args:
        cli_command: list string, CLI command to execute
        
    Returns:
        list of outputs from executing each CLI command
    """
    result = []
    device_parameters = {
       'device_type': 'cisco_ios',
       'ip': HOST,
       'username': USERNAME,
       'password': PASSWORD,
       'secret': ENABLE_PASSWORD
       } 
    # Connecting via NETMIKO SSH using defined parameters
    device_connection = ConnectHandler(**device_parameters)
    device_connection.find_prompt()
    for cli_command in cli_command_list:
        cli_output = device_connection.send_command(cli_command)
        print('='*40)
        print("Output from {}".format(cli_command))
        print('='*40)
        print(cli_output)
        result.append(cli_output)
    device_connection.disconnect()
    return result

command_list = ["show version", "show inventory", "show ip int brief"]
templates = ["cisco_ios_show_version.template",
             "cisco_ios_show_inventory.template",
             "cisco_ios_show_ip_int_brief.template"]

outputs = get_output(command_list)


NetMikoTimeoutException: Connection to device timed-out: cisco_ios 172.20.20.10:22

We can now try parsing the data using regular expressions or use TextFSM module, which also uses regular expression templates

In [ ]:
import os
import textfsm


def parse_cli_outputs(cli_outputs, templates, command_list, show_template=False):
    """Parses CLI outputs using TextFSM templates
    
    Args:
        cli_outputs: list of strings containing each CLI output
        templates: list of strings - file names of templates which are stored in templates/ directory
        command_list: list of strings - CLI commands
        show_template: boolean - if template should be printed
    
    Returns:
        dictionary with parsed results
    """
    parsed_results = {}
    for i, template in enumerate(templates):
        with open(os.path.join('templates/', template)) as f:
            fsm_template = textfsm.TextFSM(f)
            if show_template:
                print("{delim}\nTextFSM template:\n{delim}\n{template}\n{delim}".format(
                    template=fsm_template, delim='='*40))
            fsm_result_lists = fsm_template.ParseText(cli_outputs[i])
            parsed_command_output = []
            for fsm_result_list in fsm_result_lists:
                # converting list to dictionary, fsm_template.header contains variable names
                fsm_result = {key:value for key, value in zip(fsm_template.header, fsm_result_list)}
                parsed_command_output.append(fsm_result)
            key = command_list[i]
            parsed_results[key] = parsed_command_output
    return parsed_results
with open(os.path.join('templates/', 'cisco_ios_show_inventory.template')) as f:
    print("{delim}\nshow inventory textfsm template:\n{delim}\n{template}\n{delim}".format(
                    template=f.read(), delim='='*40))
print("Parsed results:")
parsed_results_r1 = parse_cli_outputs(outputs, templates, command_list)
pprint(parsed_results_r1)
        

Let's say we would like to find the hostname of the device having a particular serial number part.  
Because we have outputs only from one device, we will cheat a little and will create a copy of the outputs and change some values

In [ ]:
from copy import deepcopy
parsed_results_r2 = deepcopy(parsed_results_r1)
parsed_results_r2['show version'][0]['hostname'] = "R2"
parsed_results_r2['show version'][0]['uptime'] = "25 minutes"
parsed_results_r2['show ip int brief'][0]['ip address'] = "10.2.3.4"
parsed_results_r2['show ip int brief'][0]['l2_link_status'] = "down"
parsed_results_r2['show inventory'][0]['serial_number'] = "0C8B4CC6AB73"

parsed_results = [parsed_results_r1, parsed_results_r2]


In [ ]:
SERIAL_NUMBER = "0C8B4CC6AB73"
#SERIAL_NUMBER = "1231234545"
def find_serial_number(parsed_results, serial_number):
    """Looks for a serial number and returns the hostname of the device having this s/n
    
    Args:
        parsed_results: list of parsed results from parse_cli_outputs function
        serial_number: string, serial number we are looking for
        
    Returns:
        string - hostname of the device having this serial number
    """
    for parsed_result in parsed_results:
        for submodule in parsed_result["show inventory"]:
            if submodule['serial_number'] == serial_number:
                hostname = parsed_result['show version'][0]['hostname'] 
                pid_description = submodule['description']
                return (hostname, pid_description)
    
match = find_serial_number(parsed_results, SERIAL_NUMBER)
if match:
    hostname, pid_description = match
    print("Device with hostname {} has part {} with S/N: {}".format(hostname, pid_description, SERIAL_NUMBER))
else:
    print("Sorry, this S/N has not been found")



Let's take a little bit more complicated CLI output, for example "show ip route"

In [ ]:
command_list = ["show ip route"]
templates = ["cisco_ios_show_ip_route.template"]

outputs = get_output(command_list)

It is still possible to parse this output using regular expressions, however they tend to become complex and unreadable:

In [ ]:
parsed_results = parse_cli_outputs(outputs, templates, command_list, show_template=True)
pprint(parsed_results)

### Introducing NETCONF/YANG
Let's take a look how we can retrieve this information using NETCONF/YANG:

In [ ]:
import ncclient.manager
import xml.dom.minidom
from ncclient.operations import TimeoutExpiredError

nckwargs = {
    'host': HOST,
    'username': USERNAME,
    'password': PASSWORD,
    'device_params': {'name':"csr"}
    }

m = ncclient.manager.connect(**nckwargs) 

def prettify_xml(xml_string):
    xmlDom = xml.dom.minidom.parseString(str(xml_string))
    return xmlDom.toprettyxml(indent=" ")


In [ ]:
route_filter = '''
  <routing-state xmlns="urn:ietf:params:xml:ns:yang:ietf-routing">
    <routing-instance>
      <ribs>
        <rib>
          <name>ipv4-default</name>
          <address-family xmlns:v4ur="urn:ietf:params:xml:ns:yang:ietf-ipv4-unicast-routing">v4ur:ipv4-unicast</address-family>
          <default-rib>true</default-rib>
          <routes/>
        </rib>
      </ribs>
    </routing-instance>
  </routing-state>
  '''

try:
    xml_result = m.get(filter=('subtree', route_filter))
    print(prettify_xml(xml_result))
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print(e)

Now that is a lot of information to look through, maybe you are only interested in a few pieces of information per route. To filter XML we can use XPATH, for example:

In [ ]:
import pprint
ns = 'urn:ietf:params:xml:ns:yang:ietf-routing'
nsmap = dict(rt=ns)
route_table=[]
routes = xml_result.data.findall(".//{%s}route" % ns)
for i, route in enumerate(routes):
    dest = route.xpath('./rt:destination-prefix/text()', namespaces=nsmap)
    hop = route.xpath('./rt:next-hop/rt:next-hop-address/text()', namespaces=nsmap)
    intfc = route.xpath('./rt:next-hop/rt:outgoing-interface/text()', namespaces=nsmap)
    prot = route.xpath('./rt:source-protocol/text()', namespaces=nsmap)
    if len(hop) > 0:
        entry = {'hop':hop[0]}
    if len(intfc) > 0:
        entry = {'intfc':intfc[0]}  
    entry.update({'dest':dest[0], 'prot':prot[0]}) 
    route_table.append(entry)
pp = pprint.PrettyPrinter(indent=2)
pp.pprint(route_table)  

That seems easier to work with, but what are the items we can request? This is defined by YANG models.  
There are two types of models, native (vendor-specific) or the common models like ietf or openconfig.  
For the next example, we will download and explore IETF-interfaces model:

In [ ]:
from subprocess import Popen, PIPE, STDOUT
SCHEMA_TO_GET = 'ietf-interfaces'
print('We are connecting to the device and downloading the YANG Module')
print("Now we will do a 'pyang -f tree of %s.yang'" % SCHEMA_TO_GET)
print('This structure is what data we can get (ro/rw) or set (rw)')
schema = m.get_schema(SCHEMA_TO_GET)
p = Popen(['pyang', '-f', 'tree'], stdout=PIPE, stdin=PIPE, stderr=PIPE)
stdout_data = p.communicate(input=schema.data.encode())[0].decode()
print(stdout_data)

We can also poll other operational data like CPU utilization. In this case we are using vendor-specific model:

In [ ]:
CPU_UTIL_FILTER = '''
<cpu-usage xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-process-cpu-oper">
<cpu-utilization>
<cpu-usage-processes>
</cpu-usage-processes>
</cpu-utilization>
</cpu-usage>
'''

xml_result = m.get(filter=('subtree', CPU_UTIL_FILTER))
print(prettify_xml(xml_result))

Let's convert the output to dictionary using xmltodict and show processes only if their one minute CPU utilization is more than 0%:

In [ ]:
import xmltodict
cpu_dict = xmltodict.parse(str(xml_result))['rpc-reply']['data']
for process in cpu_dict['cpu-usage']['cpu-utilization']['cpu-usage-processes']['cpu-usage-process']:
    process_name = process['name']
    process_one_minute_utilization = float(process['one-minute'])
    if process_one_minute_utilization > 0:
        print("Process name: {}, one minute CPU utilization: {}%".format(process_name, process_one_minute_utilization))


Let's now take a look on retrieving interface statistics using standard IETF model. On top of that, we would like to see the ratio of input errors to total number of packets:


In [ ]:
INT_STATS_FILTER ='''
<filter>
  <interfaces-state xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
    <interface>
      <name></name>
      <statistics>
        <in-octets />
        <in-unicast-pkts />
        <in-errors />
      </statistics>
    </interface>
  </interfaces-state>
</filter>'''
try:
    int_stats_xml = m.get(INT_STATS_FILTER)
except:
    m = ncclient.manager.connect(**nckwargs)
    int_stats_xml = m.get(INT_STATS_FILTER)
    
print(prettify_xml(int_stats_xml))

int_stats_dict = xmltodict.parse(str(int_stats_xml))['rpc-reply']['data']
for interface in int_stats_dict['interfaces-state']['interface']:
    unicast_packets = int(interface['statistics']['in-unicast-pkts'])
    if unicast_packets == 0:
        input_errors_rate = 0
    else:
        input_errors = int(interface['statistics']['in-errors'])
        input_errors_rate = input_errors / unicast_packets
    int_name = interface['name']
    print("Interface {} has {} input unicast packets, {} input errors which result in input errors rate: {:.2%}".format(int_name, unicast_packets, input_errors, input_errors_rate))


Let's also retrieve configuration for interfaces (defined in IETF model):

In [ ]:
INT_CONF_FILTER ='''
<filter>
  <interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
    <interface></interface>
  </interfaces>
</filter>'''
try:
    int_conf_xml = m.get_config('running', INT_CONF_FILTER)
except:
    m = ncclient.manager.connect(**nckwargs)
    int_conf_xml = m.get_config('running', INT_CONF_FILTER)
    
print(prettify_xml(int_conf_xml))

## YDK
Working with XML is fine, but there is another approach. What if the model is converted to OOP (object-oriented programming) objects native to the language (in our case it is Python).  
Meet **YDK** - models are compiled to Python objects.  
Let's take a look how to work with YDK.

In [ ]:
from ydk.services import CRUDService
from ydk.providers import NetconfServiceProvider
from ydk.models.ietf import ietf_interfaces as intf
from ydk.models.ietf import iana_if_type as iftype

# create NETCONF provider
provider = NetconfServiceProvider(address=HOST,
                                  username=USERNAME,
                                  password=PASSWORD)

# create CRUD service
crud = CRUDService()


In [ ]:
# query object
query = intf.Interfaces()

# get stuff
intfs = crud.read(provider, query)

# print interface names and types
for i in intfs.interface:
    print("Interface: {}, type: {}, description: {}".format(i.name, i.type._meta_info().yang_name, i.description))

What about retrieving interface statistics?

In [ ]:
query = intf.InterfacesState()

# retrieve operational data
interfaces = crud.read(provider, query)
for interface in interfaces.interface:
    print("Interface: {}, Output packets: {}, operational status: {}".format(
        interface.name, interface.statistics.out_unicast_pkts, interface.oper_status))



### Create Loopback using YDK

In [ ]:
# Change to your name
AUTHOR = "Attendee"
# new interface
new_loopback = intf.Interfaces.Interface()

# create a new loopback
new_loopback.name = "Loopback2449"
new_loopback.type = iftype.SoftwareloopbackIdentity()
new_loopback.description = "Created by {} during DEVNET-2449 at Cisco Live US 2017".format(AUTHOR)
res = crud.create(provider, new_loopback)

Let's check if it worked:

In [ ]:
query = intf.Interfaces()
intfs = crud.read(provider, query)

for i in intfs.interface:
    print("Interface: {}, type: {}, description: {}".format(i.name, i.type._meta_info().yang_name, i.description))

### Delete Loopback using YDK

In [ ]:
# create CRUD service
crud = CRUDService()

# interface to delete
interface_to_delete = intf.Interfaces.Interface()

interface_to_delete.name = 'Loopback2449'
res = crud.delete(provider, interface_to_delete)

Let's check again:

In [ ]:
query = intf.Interfaces()
intfs = crud.read(provider, query)

for i in intfs.interface:
    print("Interface: {}, type: {}, description: {}".format(i.name, i.type._meta_info().yang_name, i.description))